## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from tqdm import tqdm
import wandb

## Hyperparameter Tuning using W&B Sweep

In [2]:
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [4]:
parameters_dict = {
    'optimizer': {
        'values': ['adam']
        }
    }

sweep_config['parameters'] = parameters_dict

In [5]:
parameters_dict.update({
    'epochs': {
        'value': 10}
    })

In [6]:
parameters_dict.update({
    'learning_rate': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

In [7]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'epochs': {'value': 10},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam']}}}


In [8]:
sweep_id = wandb.sweep(sweep_config, project="AgeEstimation-sweep")

Create sweep with ID: 1l6lya0w
Sweep URL: https://wandb.ai/kiaeipour/AgeEstimation-sweep/sweeps/1l6lya0w


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        train,test = build_dataset(config.batch_size)
        network = build_network()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            loss = train_epoch(network, train, optimizer,epoch)
            wandb.log({'epochs':  epoch + 1,'loss': loss})

## Load Dataset

In [10]:
class CustomDataset(Dataset):
    def __init__(self,image,label,transform):
        super().__init__()
        self.image = image
        self.label = label
        self.transform = transform
        
    def __getitem__(self,index):
        label = self.label[index]
        image = self.image[index]
        image = self.transform(image)
        
        return image, label
    
    def __len__(self):
        return len(self.label)

In [11]:
 def build_dataset(batch_size):
    images = []
    ages = []
    for image in os.listdir("../input/utkface-new/crop_part1"):
        age = int(image.split("_")[0])
        ages.append(age)
        img = cv2.imread(f"../input/utkface-new/crop_part1/{image}")
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        images.append(img)
        
    images = pd.Series(images,name="Images")
    ages = pd.Series(ages,name="Ages")
    df = pd.concat([images,ages],axis=1)
    under_4 = df[df["Ages"]<=4]
    under_4_new = under_4.sample(frac=0.3)
    up_4 = df[df["Ages"]>4]
    df = pd.concat([under_4_new,up_4],axis=0)
    df = df[df["Ages"]<90]
    
    X = np.array(df["Images"].tolist())
    Y = np.array(df["Ages"].tolist())
    X = X.reshape((-1, 3, X.shape[1], X.shape[2]))
    Y = Y.reshape(Y.shape[0], 1)
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    
    transform = torchvision.transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((50,50)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    
    dataset = CustomDataset(X,Y,transform)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset,[train_size,test_size])
    train = torch.utils.data.DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=4)
    test = torch.utils.data.DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=4)
    
    return train, test

## Transfer Learning

### Resnet152

In [12]:
def build_network():
    model = torchvision.models.resnet152(pretrained=True)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features,1)
    model = model.to(device)
    ct = 0
    for child in model.children():
        ct += 1
        if ct < 7:
            for param in child.parameters():
                param.requires_grad = False
                
    return model

In [13]:
def build_optimizer(model, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
    return optimizer

In [14]:
loss_function = nn.MSELoss()

In [15]:
def train_epoch(model, train, optimizer,epoch):
    train_loss = 0.0
    model.train()
    for image, label in tqdm(train):
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        predict = model(image.float())
        loss = loss_function(predict,label.float())
        loss.backward()
        optimizer.step()

        train_loss += loss

    train_total_loss = train_loss / len(train)

    print(f"Epochs:{epoch+1}, Loss:{train_total_loss}")
    
    return train_total_loss

In [16]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: ceazvq04 with config:
wandb: 	batch_size: 208
wandb: 	epochs: 10
wandb: 	learning_rate: 0.07357557081520431
wandb: 	optimizer: adam
wandb: Currently logged in as: kiaeipour. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

100%|██████████| 205/205 [00:33<00:00,  6.10it/s]


Epochs:1, Loss:369.794677734375


...

100%|██████████| 205/205 [00:26<00:00,  7.67it/s]


Epochs:9, Loss:64.63326263427734


100%|██████████| 205/205 [00:26<00:00,  7.71it/s]


Epochs:10, Loss:59.11389923095703


epochs,▁▂▃▃▄▅▆▆▇█
loss,█▅▄▃▃▂▂▁▁▁
epochs,10
loss,59.1139
